In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import datetime
from tensorflow.keras import utils
from keras.layers import Dense, Activation, Flatten, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential

In [19]:
data_2021 = pd.read_csv("données/valeursfoncieres-2022-s1.csv", sep="|")

C:\Users\huso\AppData\Local\Temp\ipykernel_19676\2718744659.py:1: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2021 = pd.read_csv("données/valeursfoncieres-2022-s1.csv", sep="|")


In [20]:
data_2021

,Code service sages,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,1,2.0,Appartement,NaN,24.0,1.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,84.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,88.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,1,2.0,Appartement,NaN,140.0,3.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,2,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1429089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1429090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30/06/2022,Vente,...,NaN,1,2.0,Appartement,NaN,15.0,1.0,NaN,NaN,NaN
1429091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10/02/2022,Adjudication,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN


In [21]:
data_2021_c = data_2021[['Date mutation', 'Nature mutation', 'Valeur fonciere', 'No voie', 'Code postal',
                         'Code departement', 'Code type local', 'Code voie', 'Code type local', 'Surface reelle bati']]

In [22]:
data_2021_c.head()

,Date mutation,Nature mutation,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code voie,Code type local,Surface reelle bati
0,03/01/2022,Vente,"55000,00",13.0,1000.0,1,2.0,2280,2.0,24.0
1,03/01/2022,Vente,"143000,00",NaN,1480.0,1,NaN,B010,NaN,NaN
2,03/01/2022,Vente,"143000,00",NaN,1480.0,1,NaN,B010,NaN,NaN
3,03/01/2022,Vente,"143000,00",98.0,1480.0,1,2.0,0055,2.0,140.0
4,04/01/2022,Vente,"300,00",NaN,1480.0,1,NaN,B031,NaN,NaN


In [23]:
data_2021_c.dropna(axis=0, inplace=True)
data_2021_c= data_2021_c.fillna(0)
data_2021_c = data_2021_c[data_2021_c['Nature mutation'] == 'Vente']
data_2021_c = data_2021_c.drop(columns='Nature mutation')

C:\Users\huso\AppData\Local\Temp\ipykernel_19676\3405295925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2021_c.dropna(axis=0, inplace=True)


In [24]:
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(int)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(int)
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(str)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(str)
data_2021_c["ID_Location"] = data_2021_c['Code voie'].astype(str) +"-"+ data_2021_c["Code postal"]+"-"+ data_2021_c["No voie"]
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(float)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(float)

In [25]:
data_2021_c['Valeur fonciere'] = data_2021_c['Valeur fonciere'].str.replace(',', '.')
data_2021_c['Valeur fonciere'] = data_2021_c['Valeur fonciere'].astype(float)
data_2021_c['Valeur fonciere'] = data_2021_c['Valeur fonciere'].astype(int)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(int)
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(int)
data_2021_c['Code type local'] = data_2021_c['Code type local'].astype(int)
#data_2021_c['Surface terrain'] = data_2021_c['Surface terrain'].astype(int)
data_2021_c['Surface reelle bati'] = data_2021_c['Surface reelle bati'].astype(int)

In [26]:
data_2021_c.set_index('ID_Location', inplace=True)

In [27]:
data_2021_c

,Date mutation,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code voie,Code type local,Surface reelle bati
ID_Location,,,,,,,,,
2280-1000-13,03/01/2022,55000,13,1000,1,2,2280,2,24
0055-1480-98,03/01/2022,143000,98,1480,1,2,0055,2,140
0130-1560-282,06/01/2022,255000,282,1560,1,1,0130,1,108
0300-1390-217,03/01/2022,525000,217,1390,1,2,0300,2,126
0300-1390-217,03/01/2022,525000,217,1390,1,4,0300,4,424
...,...,...,...,...,...,...,...,...,...
3536-75015-43,21/06/2022,423000,43,75015,75,2,3536,2,43
3536-75015-43,21/06/2022,423000,43,75015,75,3,3536,3,0
3536-75015-43,21/06/2022,423000,43,75015,75,3,3536,3,0


In [28]:
data_2021_c.sort_values(by=['ID_Location'], ascending=True, inplace=True)

In [29]:
data_2021_c = data_2021_c.drop_duplicates()
data_2021_c

,Date mutation,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code voie,Code type local,Surface reelle bati
ID_Location,,,,,,,,,
0001-10190-46,31/01/2022,235000,46,10190,10,1,0001,1,106
0001-10200-21,14/01/2022,90000,21,10200,10,3,0001,3,0
0001-10200-21,14/01/2022,90000,21,10200,10,1,0001,1,128
0001-10200-45,24/02/2022,80800,45,10200,10,1,0001,1,96
0001-10200-9,09/02/2022,176500,9,10200,10,3,0001,3,0
...,...,...,...,...,...,...,...,...,...
X395-83270-265,04/02/2022,1441400,265,83270,83,1,X395,1,182
X395-83270-265,04/02/2022,1441400,265,83270,83,3,X395,3,0
X642-75012-6,25/03/2022,917000,6,75012,75,3,X642,3,0


In [30]:
data = data_2021_c.groupby(['ID_Location', 'Date mutation'])

In [31]:
print(data.head(5))

               Date mutation  Valeur fonciere  No voie  Code postal  \
ID_Location                                                           
0001-10190-46     31/01/2022           235000       46        10190   
0001-10200-21     14/01/2022            90000       21        10200   
0001-10200-21     14/01/2022            90000       21        10200   
0001-10200-45     24/02/2022            80800       45        10200   
0001-10200-9      09/02/2022           176500        9        10200   
...                      ...              ...      ...          ...   
X395-83270-265    04/02/2022          1441400      265        83270   
X395-83270-265    04/02/2022          1441400      265        83270   
X642-75012-6      25/03/2022           917000        6        75012   
X642-75012-6      25/03/2022           917000        6        75012   
X642-75012-6      24/06/2022            30000        6        75012   

               Code departement  Code type local Code voie  Code type local 

In [ ]:
train, test = train_test_split(data_2021_c, test_size=0.2, shuffle=False)

In [ ]:
window_size = 20 # nombre de lignes à utiliser pour prédire la valeur foncière
train_generator = TimeseriesGenerator(train[['Surface terrain', 'Surface reelle bati']].values, train['Valeur fonciere'].values, length=window_size, batch_size=1000)
test_generator = TimeseriesGenerator(test[['Surface terrain', 'Surface reelle bati']].values, test['Valeur fonciere'].values, length=window_size, batch_size=1000)

In [40]:
len(train_generator)

357

In [41]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window_size, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics='acc')

In [ ]:
model.fit(train_generator, epochs=10, validation_data=(test_generator))

Epoch 1/10
250/357 [====================>.........] - ETA: 4s - loss: 4143470346240.0000 - acc: 0.0031